##### Imports

In [189]:
import numpy as np
import pandas as pd
import os
import time
from itertools import permutations, combinations

##### Prompts to choose which store you want 

In [190]:
print("Welcome to Apriori 2.0!")
store_num = input("Please select your store \n 1. Amazon \n 2. Nike \n 3. Best Buy \n 4. K-Mart \n 5. Walmart\n")
print(store_num)
support_percent = input("Please enter the percentage of Support you want?\n")
print(support_percent)
confidence_percent = input("Please enter the percentage of Confidence you want?\n")
print(confidence_percent)

Welcome to Apriori 2.0!
Please select your store 
 1. Amazon 
 2. Nike 
 3. Best Buy 
 4. K-Mart 
 5. Walmart
1
1
Please enter the percentage of Support you want?
23
23
Please enter the percentage of Confidence you want?
23
23


##### These are my dictionaries to choose which store to get based in Key-Value Pairs

In [191]:
def number_to_store(store_number):
    switcher = {
        1: "data/amazon_transactions.csv",
        2: "data/nike_transaction.csv",
        3: "data/best_buy_transaction.csv",
        4: "data/k_mart_transaction.csv",
        5: "data/walmart_transaction.csv"
    }
    return switcher.get(store_number)


def number_to_item_list_of_store(store_number):
    switcher_dict = {
        1: "data/amazon_item_names.csv",
        2: "data/nike_item_names.csv",
        3: "data/best_buy_item_names.csv",
        4: "data/k_mart_item_names.csv",
        5: "data/walmart_item_names.csv"
    }
    return switcher_dict.get(store_number)

##### We first have to read in the csv files and make sure that the inputs received from the user are valid

In [192]:
def a_priori_read(item_list, transaction, support_percentage, confidence_percentage):
    # Create two different functions one that is solo for read in the file data and the other that is algorithmic with the data
    if support_percentage > 100 or confidence_percentage > 100 or support_percentage < 0 or confidence_percentage < 0:
        print("Support Percent or Confidence Percent is Invalid. \n Enter a valid number between 0 and 100.\n")
        print("Restarting Apriori 2.0.....\n")
        time.sleep(2)
        os.system("python APrior_2.0.py")
    if support_percentage >= 0 and support_percentage <= 100 and confidence_percentage >= 0 and confidence_percentage <= 100:
        df2 = pd.read_csv(item_list)
        df1 = pd.read_csv(transaction)
        print(df1.head())
        print(df2.head())
        trans = np.array(df1["transaction"])
        items_names = np.array(df2["item_name"])
        k_value = 1
        return items_names, trans, support_percentage, confidence_percentage, k_value

##### The first go around the apriori algorithm we find the items that are most frequent K = 1

In [193]:
def ap_1(items_names, trans, support_percentage, confidence_percentage, k_value):
    counter = np.zeros(len(items_names), dtype=int)
    for i in trans:
        i = list((map(str.strip, i.split(','))))
        s1 = set(i)
        nums = 0
        for x in items_names:
            s2 = set()
            s2.add(x)
            if s2.issubset(s1):
                counter[nums] += 1
            nums += 1
    counter = list(map(lambda x: int((x / len(trans)) * 100), counter))
    df3 = pd.DataFrame({"item_name": items_names, "support": counter,"k_val" : np.full(len(items_names),k_value)})
    rslt_df = df3[df3['support'] > support_percentage]
    print("When K = " + str(k_value))
    print(rslt_df)
    items = np.array(rslt_df["item_name"])
    support_count = np.array(rslt_df["support"])
    k_value += 1
    return items, support_count, k_value, rslt_df

##### Then we use the second function to find item sets that are most frequent when K > 1

In [194]:
def ap_2(item_comb, k_value, trans, support_percentage):
    boo = True
    comb = combinations(item_comb, k_value)
    comb = list(comb)
    #print(comb)
    counter = np.zeros(len(comb), dtype=int)
    #print(counter)
    if k_value > 1:
        for i in trans:
            i = list((map(str.strip, i.split(','))))
            s1 = set(i)
            nums = 0
            for x in comb:
                s2 = set()
                x = np.asarray(x)
                for q in x:
                    s2.add(q)
                if s2.issubset(s1):
                    counter[nums] += 1
                nums += 1
    counter = list(map(lambda x: int((x / len(trans)) * 100), counter))
    df3 = pd.DataFrame({"item_name": comb, "support": counter,"k_val":np.full(len(comb),k_value)})
    rslt_df = df3[df3['support'] > support_percentage]
    print("When K = " + str(k_value))
    print(rslt_df)
    items = np.array(rslt_df["item_name"])
    supp = np.array(rslt_df["support"])
    if len(items) == 0:
        #print("hello")
        boo = False
        # calculate confidence
        return rslt_df, boo
        # exit(12321)
    # return items, supp, k_value
    return rslt_df, boo

##### Calls of functions and variable saving

In [195]:
frames = []
items_names, trans, support_percent, confidence_percent, k_value = a_priori_read(
    str(number_to_item_list_of_store(int(store_num))), str(number_to_store(int(store_num))),
    int(support_percent), int(confidence_percent))

items, supp, k_value, df = ap_1(items_names, trans, support_percent, confidence_percent, k_value)
frames.append(df)
boo = True

   transaction_id                                        transaction
0               1  A Beginner’s Guide, Java: The Complete Referen...
1               2  A Beginner’s Guide, Java: The Complete Referen...
2               3  A Beginner’s Guide, Java: The Complete Referen...
3               4  Android Programming: The Big Nerd Ranch, Head ...
4               5  Android Programming: The Big Nerd Ranch, Begin...
   item_number                                item_name
0            1                       A Beginner’s Guide
1            2             Java: The Complete Reference
2            3                         Java For Dummies
3            4  Android Programming: The Big Nerd Ranch
4            5              Head First Java 2nd Edition
When K = 1
                                 item_name  support  k_val
0                       A Beginner’s Guide       55      1
1             Java: The Complete Reference       50      1
2                         Java For Dummies       65      1
3  

##### Looping through the transactions and increasing K until we can longer support the support value

In [196]:
while boo:
    df_1, boo = ap_2(items, k_value, trans, support_percent)
    #print("here is df")
    #print(df_1)
    frames.append(df_1)
    k_value += 1

When K = 2
                                            item_name  support  k_val
0   (A Beginner’s Guide, Java: The Complete Refere...       45      2
1              (A Beginner’s Guide, Java For Dummies)       45      2
2   (A Beginner’s Guide, Android Programming: The ...       30      2
5    (Java: The Complete Reference, Java For Dummies)       50      2
6   (Java: The Complete Reference, Android Program...       30      2
9   (Java For Dummies, Android Programming: The Bi...       45      2
12  (Android Programming: The Big Nerd Ranch, Head...       30      2
When K = 3
                                            item_name  support  k_val
0   (A Beginner’s Guide, Java: The Complete Refere...       45      3
1   (A Beginner’s Guide, Java: The Complete Refere...       25      3
4   (A Beginner’s Guide, Java For Dummies, Android...       25      3
10  (Java: The Complete Reference, Java For Dummie...       30      3
When K = 4
                                           item_name  sup

In [197]:
print("results are below")
print(pd.concat(frames))
df_supp = pd.concat(frames)

results are below
                                            item_name  support  k_val
0                                  A Beginner’s Guide       55      1
1                        Java: The Complete Reference       50      1
2                                    Java For Dummies       65      1
3             Android Programming: The Big Nerd Ranch       65      1
4                         Head First Java 2nd Edition       40      1
5                     Beginning Programming with Java       30      1
0   (A Beginner’s Guide, Java: The Complete Refere...       45      2
1              (A Beginner’s Guide, Java For Dummies)       45      2
2   (A Beginner’s Guide, Android Programming: The ...       30      2
5    (Java: The Complete Reference, Java For Dummies)       50      2
6   (Java: The Complete Reference, Android Program...       30      2
9   (Java For Dummies, Android Programming: The Bi...       45      2
12  (Android Programming: The Big Nerd Ranch, Head...       30      2
0 

In [198]:
df_supp = df_supp.reset_index().drop('index',axis=1)

In [199]:
df_supp

,item_name,support,k_val
0,A Beginner’s Guide,55,1
1,Java: The Complete Reference,50,1
2,Java For Dummies,65,1
3,Android Programming: The Big Nerd Ranch,65,1
4,Head First Java 2nd Edition,40,1
5,Beginning Programming with Java,30,1
6,"(A Beginner’s Guide, Java: The Complete Refere...",45,2
7,"(A Beginner’s Guide, Java For Dummies)",45,2
8,"(A Beginner’s Guide, Android Programming: The ...",30,2
9,"(Java: The Complete Reference, Java For Dummies)",50,2


##### Support of single Items

In [200]:
def confidence(val):
    #print(val)
    #first
    df_212 = df_supp.loc[df_supp['k_val'] == val] 
    stuff_name = np.array(df_212["item_name"])
    support_arr = np.array(df_212['support'])
    
    #second
    df_temp = df_supp.loc[df_supp['k_val'] == val+1] 
    df_21231 = np.array(df_temp["item_name"])
    sup_arr = np.array(df_temp['support'])
    
    #variables to save
    sup_ov = list()
    sup_sing = list()
    perm_item = list()
    if val == 1:
        for w in df_21231:
            temp_list = list(df_21231)
    #     print(len(w))
    #     print("this is the support val")
            ov_sup = sup_arr[temp_list.index(w)]
            temp = set()
            for d in w:
                temp.add(d)
           # print(temp)
            perm = permutations(temp)
            perm_lst = list(perm)
            #print(perm_lst)
            for y in perm_lst:
                #print(y)
                perm_item.append(y)
                sup_ov.append(ov_sup)
                sup_sing.append(int(support_arr[np.where(stuff_name == y[0])]))
    if val > 1:    
        for w in df_21231:
            temp_list = list(df_21231)
            ov_sup = sup_arr[temp_list.index(w)]
            #print("OV_SUPPORT")
            #print(ov_sup)
            temp = set()
            for d in w:
                temp.add(d)
            perm = permutations(temp)
            perm_lst = list(perm)
            #print(len(perm_lst))
            for y in perm_lst:
                try:
                   # print("PERM_ITEM")
                    #print(y)
                    #print("-------------")
                    temp_set = []
                   # perm_item.append(y)
                    #sup_ov.append(ov_sup)
                    for t in range(0,val):
                        #print(t)
                        temp_set.append(y[t])
                   # print("this is SUP_SING")
                    lol = tuple(sorted(temp_set))
                    #print(lol)
                    tp_lst = list(stuff_name)
                    #print("SUP_SING")
                    ss = support_arr[tp_lst.index(lol)]
                    sup_ov.append(ov_sup)
                    sup_sing.append(ss)
                    perm_item.append(y)
                    #print(ss)
                except:
                    #print("itemset below does not exist...")
                    #print(y)
                    sup_ov.append(ov_sup)
                    sup_sing.append(0)
                    perm_item.append(y)
#             print(len(sup_ov))
#             print(len(sup_sing))
#             print(len(perm_item))
#             print(sup_ov)
#             print(sup_sing)
#             print(perm_item)
    df_main = pd.DataFrame({"association":perm_item,"support_ov":sup_ov,"support_sing":sup_sing})
    df_main = df_main.assign(confidence = lambda x:round(((x.support_ov/x.support_sing)*100),0))
    return df_main

In [201]:
max(df_supp["k_val"])

4

In [202]:
df_frames = []
for lp in range(1,max(df_supp["k_val"])+1):
    print(lp)
    df_0 = confidence(lp)
    df_0 = df_0[df_0.support_sing != 0]
    df_frames.append(df_0)

print(pd.concat(df_frames))
df_associations = pd.concat(df_frames)
    
# df_0 = confidence(1)

1
2
3
4
                                          association  support_ov  \
0   (Java: The Complete Reference, A Beginner’s Gu...        45.0   
1   (A Beginner’s Guide, Java: The Complete Refere...        45.0   
2              (A Beginner’s Guide, Java For Dummies)        45.0   
3              (Java For Dummies, A Beginner’s Guide)        45.0   
4   (A Beginner’s Guide, Android Programming: The ...        30.0   
5   (Android Programming: The Big Nerd Ranch, A Be...        30.0   
6    (Java: The Complete Reference, Java For Dummies)        50.0   
7    (Java For Dummies, Java: The Complete Reference)        50.0   
8   (Java: The Complete Reference, Android Program...        30.0   
9   (Android Programming: The Big Nerd Ranch, Java...        30.0   
10  (Android Programming: The Big Nerd Ranch, Java...        45.0   
11  (Java For Dummies, Android Programming: The Bi...        45.0   
12  (Head First Java 2nd Edition, Android Programm...        30.0   
13  (Android Programming: 

In [204]:
df_associations

,association,support_ov,support_sing,confidence
0,"(Java: The Complete Reference, A Beginner’s Gu...",45.0,50.0,90.0
1,"(A Beginner’s Guide, Java: The Complete Refere...",45.0,55.0,82.0
2,"(A Beginner’s Guide, Java For Dummies)",45.0,55.0,82.0
3,"(Java For Dummies, A Beginner’s Guide)",45.0,65.0,69.0
4,"(A Beginner’s Guide, Android Programming: The ...",30.0,55.0,55.0
5,"(Android Programming: The Big Nerd Ranch, A Be...",30.0,65.0,46.0
6,"(Java: The Complete Reference, Java For Dummies)",50.0,50.0,100.0
7,"(Java For Dummies, Java: The Complete Reference)",50.0,65.0,77.0
8,"(Java: The Complete Reference, Android Program...",30.0,50.0,60.0
9,"(Android Programming: The Big Nerd Ranch, Java...",30.0,65.0,46.0
